In [1]:
import os
import pandas as pd
import csv

1. 基于CNCB和GISAID数据库下载中国20221201至20231031的新冠病毒基因组数据

In [ ]:
# 根据“CNCB_GISAID_metadata_china_20221201-20231031.xlsx”提供的ID从GISAID下载SARS-CoV-2基因组序列:"Gisaid-china.fasta",共41708条
# 使用nextclade工具对其进行运算，保留nextclade.csv、nextclade.tsv、nextclade.aligned.fasta.gz结果，解压.gz文件。

# cd /home/soniali/Desktop/02_china_recom_renew/0_raw_data
# nextclade run \
#    --input-dataset data/sars-cov-2 \
#    --output-all=/home/soniali/Desktop/02_china_recom_renew/0_raw_data/China_output_Gisaid \
# /home/soniali/Desktop/02_china_recom_renew/0_raw_data/Gisaid-china.fasta

In [2]:
dirpath = "/home/soniali/Desktop/02_china_recom_renew/0_raw_data/"
gisaid_metacsv = dirpath+"CNCB_GISAID_metadata_china_20221201-20231031.xlsx"
gisaid_nexttsv = dirpath+"China_output_Gisaid/nextclade.tsv"   ## 保存在14.174中/Data4/Data_public/china/output_Gisaid

2. 对原始新冠病毒基因组数据进行质量过滤，保留高质量序列和对应流行学信息

In [3]:
df0 = pd.read_table(gisaid_nexttsv)
df = df0.loc[:,["seqName","Nextclade_pango","partiallyAliased","clade_nextstrain","clade_display","qc.overallStatus","coverage"]]
print(df.shape) # (41708,7)
df1=  df.loc[df['qc.overallStatus'] == "good"] 
print(df1.shape) #(40091, 7)
df2=  df1.loc[df['coverage'] >=0.97] 
print(df2.shape) #(40091, 7)
gisaid_qualified_ID = df2.seqName.tolist()
# qualified_ID

(41708, 7)
(40091, 7)
(40091, 7)


/tmp/ipykernel_183268/464067568.py:1: DtypeWarning: Columns (55,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_table(gisaid_nexttsv)


In [5]:
gisaid_qualified_epi = []
for stain_name in gisaid_qualified_ID:
    if "EPI_ISL" in stain_name:
        seq_id = "EPI_"+stain_name.split("|EPI_")[1].split("|")[0]
        gisaid_qualified_epi.append(seq_id)

In [6]:
df_meta = pd.read_excel(gisaid_metacsv)
df_meta = pd.DataFrame(df_meta)
df_meta_2 = df_meta[df_meta["Accession ID"].isin(gisaid_qualified_epi)]
num = 0
chinese_qualified_epi = []
for i in df_meta_2.index:
    if str(df_meta_2.loc[i,"Sample Collection Date"]).count("-") >=2:
        if df_meta_2.loc[i,"Location"].count("/") >=1:
            chinese_qualified_epi.append(df_meta_2.loc[i,"Accession ID"])
            num+=1
num


39456

In [7]:
df_meta_qualified = df_meta_2[df_meta_2["Accession ID"].isin(chinese_qualified_epi)]
df_meta_qualified = df_meta_qualified.drop(columns=['Sequence Length'])

for i in df_meta_qualified.index:
    df_meta_qualified.loc[i,"country"] = df_meta_qualified.loc[i,"Location"].split("/")[0].strip()
    df_meta_qualified.loc[i,"province"] = df_meta_qualified.loc[i,"Location"].split("/")[1].strip()
    if len(df_meta_qualified.loc[i,"Location"].split("/")) >= 3:
        df_meta_qualified.loc[i,"city"] = df_meta_qualified.loc[i,"Location"].split("/")[2].strip()
    else:
        df_meta_qualified.loc[i,"city"] = ""
        
df_meta_qualified.head()

,Virus Strain Name,Accession ID,Lineage,Sample Collection Date,Location,country,province,city
0,hCoV-19/Henan/HBCDC-CX109/2023,EPI_ISL_18543055,EG.5.1.1,2023-10-26 00:00:00,China / Henan / Hebi,China,Henan,Hebi
1,hCoV-19/Henan/HBCDC-CX108/2023,EPI_ISL_18543054,HK.2,2023-10-26 00:00:00,China / Henan / Hebi,China,Henan,Hebi
2,hCoV-19/Henan/HBCDC-CX106/2023,EPI_ISL_18543053,HK.3,2023-10-26 00:00:00,China / Henan / Hebi,China,Henan,Hebi
3,hCoV-19/Henan/HBCDC-CX105/2023,EPI_ISL_18543052,HK.3.2,2023-10-26 00:00:00,China / Henan / Hebi,China,Henan,Hebi
4,hCoV-19/Henan/HBCDC-CX104/2023,EPI_ISL_18543051,HK.3,2023-10-26 00:00:00,China / Henan / Hebi,China,Henan,Hebi


In [8]:
df_meta_qualified_2 = df_meta_qualified.drop(columns=['Location'])
df_meta_qualified_2['Sample Collection Date'] = pd.to_datetime(df_meta_qualified_2['Sample Collection Date'])
df_meta_qualified_2['Sample Collection Date'] = df_meta_qualified_2['Sample Collection Date'].dt.strftime('%m/%d/%Y')
df_meta_qualified_2.columns = df_meta_qualified_2.columns.str.replace(' ', '_')
chinese_qualified_ID = df_meta_qualified_2.Accession_ID.tolist()

In [9]:
len(chinese_qualified_ID)

39456

In [10]:
from Bio import SeqIO

with open(dirpath+"Gisaid-qualified-china.fasta","a+") as f:
    with open(dirpath+"Gisaid-china.fasta","r") as h:  #China_output_Gisaid/nextclade.aligned.fasta
        for record in SeqIO.parse(h,"fasta"):
            seq_id = str(record.id)
            epi_id = "EPI_"+seq_id.split("|EPI_")[1].split("|")[0]
            if  set([epi_id]) - set(chinese_qualified_ID) == set():
                f.write(">"+str(record.id)+"\n")
                f.write(str(record.seq)+"\n")

In [ ]:
# 对Qualified-china.fasta进行nextclade运算，生成“0_raw_data/GISAID_nextclade”文件夹

# cd /home/soniali/Desktop/02_china_recom_renew/0_raw_data
# mkdir /home/soniali/Desktop/02_china_recom_renew/0_raw_data/GISAID_nextclade/
# nextclade run \
#    --input-dataset data/sars-cov-2 \
#    --output-all=/home/soniali/Desktop/02_china_recom_renew/0_raw_data/GISAID_nextclade \
# /home/soniali/Desktop/02_china_recom_renew/0_raw_data/Gisaid-qualified-china.fasta

# 保留nextclade.csv、nextclade.tsv、nextclade.auspice.json、nextclade.aligned.fasta.gz结果，解压.gz文件。
# 将nextclade.auspice.json和Qualified_china_meta_auspice.tsv文件放入 https://auspice.us 网站，与全球新冠树进行比较，下载系统发育树分析结果

In [11]:
df = pd.read_csv(dirpath+'GISAID_nextclade/nextclade.tsv', sep='\t', usecols=['seqName', 'Nextclade_pango'])
result_dict = df.set_index('seqName')['Nextclade_pango'].to_dict()
print(df.shape)
df.head()

(39456, 2)


,seqName,Nextclade_pango
0,EPI_ISL_17775792,DY.2
1,EPI_ISL_17778583,XBB.1.5
2,EPI_ISL_17778586,XBB.1.5
3,EPI_ISL_17778578,XBB.1.5
4,EPI_ISL_17778579,FL.2


In [12]:
for i in df_meta_qualified_2.index:
    temp_lin = result_dict[df_meta_qualified_2.loc[i,"Accession_ID"]]
    df_meta_qualified_2.loc[i,"Lineage"] = temp_lin

df_meta_qualified_2.head()

,Virus_Strain_Name,Accession_ID,Lineage,Sample_Collection_Date,country,province,city
0,hCoV-19/Henan/HBCDC-CX109/2023,EPI_ISL_18543055,HK.27.1,10/26/2023,China,Henan,Hebi
1,hCoV-19/Henan/HBCDC-CX108/2023,EPI_ISL_18543054,HK.2,10/26/2023,China,Henan,Hebi
2,hCoV-19/Henan/HBCDC-CX106/2023,EPI_ISL_18543053,HK.3,10/26/2023,China,Henan,Hebi
3,hCoV-19/Henan/HBCDC-CX105/2023,EPI_ISL_18543052,HK.3.2,10/26/2023,China,Henan,Hebi
4,hCoV-19/Henan/HBCDC-CX104/2023,EPI_ISL_18543051,HK.3,10/26/2023,China,Henan,Hebi


In [13]:
df_meta_qualified_2.to_csv(dirpath+"Qualified_china_meta.csv",index=False)

3. 根据nextclade工具对39,456条序列分配的Pango谱系，确定本研究的8个主谱系及其子代谱系，形成excel

In [14]:
from collections import Counter
all_lineage = df_meta_qualified_2.Lineage.tolist()
lin_counts = Counter(all_lineage)
lin_counts_dict = dict(sorted(dict(lin_counts).items(), key=lambda item: item[1], reverse=True))
lin_counts_dict

{'DY.2': 3818,
 'BF.7.14': 3729,
 'EG.5.1.1': 2967,
 'DY.4': 2538,
 'BA.5.2.48': 1966,
 'FU.1': 1924,
 'DY.1': 1592,
 'DY.3': 1481,
 'HK.3': 1100,
 'FY.3.1': 959,
 'XBB.1.16': 861,
 'XBB.1.5': 674,
 'FL.4': 669,
 'FL.2.3': 602,
 'GF.1': 592,
 'DZ.1': 580,
 'BF.7.14.1': 522,
 'BF.7.14.5': 464,
 'FL.2': 419,
 'XBB.1.9.1': 354,
 'BF.7.14.4': 352,
 'FY.3.2': 306,
 'XBB.1.16.1': 285,
 'FL.16': 264,
 'FY.3.3': 261,
 'XBB.1.42.2': 234,
 'FL.2.4': 229,
 'FE.1.1': 228,
 'EG.5.1': 220,
 'FY.3': 203,
 'FR.1.1': 191,
 'HK.3.2': 172,
 'BA.5.2.50': 161,
 'BF.7.14.6': 152,
 'FL.13.1': 152,
 'GY.1': 144,
 'EG.5.1.8': 143,
 'DZ.2': 143,
 'BA.5.2.49': 138,
 'HK.8.1': 136,
 'EG.2': 133,
 'XBB.1.16.22': 131,
 'HK.3.1': 130,
 'BA.5.2': 129,
 'FL.21.2': 127,
 'XBB.1.9.2': 120,
 'BF.7.14.7': 113,
 'FR.1': 112,
 'GW.1': 108,
 'HK.27.1': 104,
 'FL.4.11': 96,
 'XBB.1.5.15': 94,
 'DY.1.1': 94,
 'HK.32': 92,
 'HK.14': 92,
 'BQ.1.1': 92,
 'FL.4.6': 88,
 'HK.1': 86,
 'XBB.1.16.18': 86,
 'BF.7.14.3': 86,
 'HK.2': 81

In [15]:
df_meta_qualified_2['Sample_Collection_Date'] = pd.to_datetime(df_meta_qualified_2['Sample_Collection_Date'])
# 提取月份
unique_months = df_meta_qualified_2['Sample_Collection_Date'].dt.month.unique()


In [16]:
month_lin_num = {}
# 根据月份循环提取行
for month in unique_months:
    df_month = df_meta_qualified_2[df_meta_qualified_2['Sample_Collection_Date'].dt.month == month]
    print(f"Month: {month}")
    # print(df_month)
    all_lineage = df_month.Lineage.tolist()
    lin_counts = Counter(all_lineage)
    lin_counts_dict = dict(sorted(dict(lin_counts).items(), key=lambda item: item[1], reverse=True))
    month_lin_num[month] = dict(list(lin_counts_dict.items()))


Month: 10
Month: 9
Month: 4
Month: 6
Month: 5
Month: 12
Month: 8
Month: 7
Month: 3
Month: 2
Month: 1


In [17]:
num = 0
for month in month_lin_num:
    total = sum(month_lin_num[month].values())
    num+=total

num

39456

In [18]:
all_top_lin = []
for month in month_lin_num:
    all_top_lin.extend(list(month_lin_num[month].keys()))

In [19]:
month_lin_num_sort = dict(sorted(month_lin_num.items()))
month_lin_num_sort

{1: {'DY.2': 1140,
  'BF.7.14': 967,
  'DY.4': 869,
  'BA.5.2.48': 684,
  'DY.3': 506,
  'DY.1': 477,
  'DZ.1': 201,
  'BF.7.14.1': 138,
  'BF.7.14.5': 122,
  'BF.7.14.4': 111,
  'BA.5.2.50': 76,
  'DZ.2': 55,
  'BF.7.14.6': 52,
  'BA.5.2': 51,
  'BF.7.14.3': 38,
  'DY.1.1': 33,
  'BA.5.2.49': 32,
  'BA.5.2.1': 17,
  'BF.7.14.2': 16,
  'BN.1.3': 12,
  'BF.7.14.7': 12,
  'BQ.1.1': 11,
  'BA.5.1.32': 11,
  'CH.1.1.11': 10,
  'CH.1.1': 9,
  'BN.1.3.10': 5,
  'BQ.1': 4,
  'BF.7': 4,
  'FR.1': 4,
  'BN.1.5': 3,
  'CH.1.1.3': 3,
  'CM.12': 3,
  'CR.1.1': 3,
  'XBB.1': 3,
  'BA.5.1': 3,
  'BN.1.1.1': 2,
  'EJ.2': 2,
  'BA.5.2.6': 2,
  'BN.1.3.9': 2,
  'BN.1.1': 2,
  'BQ.1.2': 2,
  'XBB.1.5': 1,
  'BF.5': 1,
  'BN.1.2.7': 1,
  'XBJ.1.1': 1,
  'BR.2.1': 1,
  'BQ.1.1.31': 1,
  'EE.1': 1,
  'BQ.1.1.44': 1,
  'BA.5.2.28': 1,
  'BN.1.2.2': 1,
  'BQ.1.31': 1,
  'BQ.1.23': 1,
  'CJ.1.3': 1,
  'BA.5.2.27': 1,
  'BQ.1.1.53': 1,
  'BQ.1.1.22': 1,
  'BQ.1.1.38': 1,
  'BA.2.76': 1,
  'BQ.1.1.4': 1,
  'XBB

In [20]:
def translate_to_month_name(month_number):
    months = {
        1: "January",
        2: "February",
        3: "March",
        4: "April",
        5: "May",
        6: "June",
        7: "July",
        8: "August",
        9: "September",
        10: "October",
        11: "November",
        12: "December"
    }
    
    if month_number in months:
        return months[month_number]
    else:
        return "Invalid month number"

In [21]:
df_lin_num = pd.DataFrame(columns=["month","lineage","genome_number"])

data = []
for mon in month_lin_num_sort:
    for lin in month_lin_num_sort[mon]:
        data.append([translate_to_month_name(mon),lin,month_lin_num_sort[mon][lin]])


In [22]:
for i in range(len(data)):
    df_lin_num.loc[i] = data[i]

df_lin_num.to_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num.csv",index = False)

In [23]:
len(set(all_top_lin))

635

In [152]:
df = pd.read_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full.csv")

for i in df.index:
    lin = df.loc[i,"lineage"] 
    if "XBB.1.5" in lin:
        df.loc[i,"lineage_full_name"] = lin
        df.loc[i,"merged_lineage"] = "XBB.1.5*"
    elif "XBB.1.16" in lin:
        df.loc[i,"lineage_full_name"] = lin
        df.loc[i,"merged_lineage"] = "XBB.1.16*"
    elif "EG.5.1" in lin:
        df.loc[i,"lineage_full_name"] = "XBB.1.9.2.5.1"+lin.split("EG.5.1")[1]
        df.loc[i,"merged_lineage"] = "EG.5.1*"
    elif ("EG.5.1" not in lin) and ("EG.5" in lin):
        df.loc[i,"lineage_full_name"] = "XBB.1.9.2.5"+lin.split("EG.5")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "FL." in lin:
        df.loc[i,"lineage_full_name"] = "XBB.1.9.1."+lin.split("FL.")[1]
        df.loc[i,"merged_lineage"] = "XBB.1.9.1*"
    elif "FY.3" in lin:
        df.loc[i,"lineage_full_name"] = "XBB.1.22.1.3"+lin.split("FY.3")[1]
        df.loc[i,"lineage_full_name"] = "FY.3*"
    elif ("FY.3" not in lin) and ("FY." in lin):
        df.loc[i,"lineage_full_name"] = "XBB.1.22.1."+lin.split("FY.")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "HK." in lin:
        df.loc[i,"lineage_full_name"] = "XBB.1.9.2.5.1.1."+lin.split("HK.")[1]
        df.loc[i,"merged_lineage"] = "EG.5.1*"


df.to_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full2.csv",index = None)

In [155]:
df = pd.read_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full2.csv")

for i in df.index:
    lin = df.loc[i,"lineage"] 
    if "BF." in lin and "XBF" not in lin:
        df.loc[i,"lineage_full_name"] = "B.1.1.529.5.2.1."+lin.split("BF.")[1]
        df.loc[i,"merged_lineage"] = "BA.5.2*"
    elif "XBF" in lin:
        df.loc[i,"lineage_full_name"] = "XBF"+lin.split("XBF")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "BQ.1" in lin:
        df.loc[i,"lineage_full_name"] = "B.1.1.529.5.3.1.1.1.1.1"+lin.split("BQ.1")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "BA.5" in lin and "BA.5.2" not in lin:
        df.loc[i,"lineage_full_name"] = "B.1.1.529.5"+lin.split("BA.5")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "BE." in lin:
        df.loc[i,"lineage_full_name"] = "B.1.1.529.5.3.1."+lin.split("BE.")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "BN." in lin:
        df.loc[i,"lineage_full_name"] = "B.1.1.529.2.75.5."+lin.split("BN.")[1]
        df.loc[i,"merged_lineage"] = "Others"

df.to_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full3.csv",index = None)

In [162]:
df = pd.read_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full3.csv")

for i in df.index:
    lin = df.loc[i,"lineage"] 
    full_lin = df.loc[i,"lineage_full_name"] 
    if "B.1.1.529.2.75" in lin:
        df.loc[i,"lineage_full_name"] = lin
        df.loc[i,"merged_lineage"] = "BA.2.75*"
    elif "CH." in lin and "XCH" not in lin:
        df.loc[i,"lineage_full_name"] = "B.1.1.529.2.75.3.4.1.1."+lin.split("CH.")[1]
        df.loc[i,"merged_lineage"] = "BA.2.75*"
    elif "CM." in lin and "XCM" not in lin:
        df.loc[i,"lineage_full_name"] = "B.1.1.529.2.3.20."+lin.split("CM.")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "DV." in lin and "XDV" not in lin:
        df.loc[i,"lineage_full_name"] = "B.1.1.529.2.75.3.4.1.1.1.1.1."+lin.split("DV.")[1]
        df.loc[i,"merged_lineage"] = "BA.2.75*"
    elif "EG." in lin and "EG.5.1" not in lin:
        df.loc[i,"lineage_full_name"] = "XBB.1.9.2.1"+lin.split("EG.")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "FK." in lin and "XFK" not in lin:
        df.loc[i,"lineage_full_name"] = "B.1.1.529.2.75.3.4.1.1.1.1.17."+lin.split("FK.")[1]
        df.loc[i,"merged_lineage"] = "BA.2.75*"
    elif "HH." in lin and "XHH" not in lin:
        df.loc[i,"lineage_full_name"] = "XBB.2.3.2."+lin.split("HH.")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "GS." in lin and "XGS" not in lin:
        df.loc[i,"lineage_full_name"] = "XBB.2.3.11."+lin.split("GS.")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "GA." in lin and "XGA" not in lin:
        df.loc[i,"lineage_full_name"] = "XBB.1.17.1."+lin.split("GA.")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "GW." in lin and "XGW" not in lin:
        df.loc[i,"lineage_full_name"] = "XBB.1.19.1."+lin.split("GW.")[1]
        df.loc[i,"merged_lineage"] = "Others"
    elif "KB."in lin and "XKB" not in lin:
        df.loc[i,"lineage_full_name"] = "XBB.1.9.2.5.1.8."+lin.split("KB.")[1]
        df.loc[i,"merged_lineage"] = "EG.5*"
    elif "JD." in lin and "XJD" not in lin:
        df.loc[i,"lineage_full_name"] = "XBB.1.5.102."+lin.split("JD.")[1]
        df.loc[i,"merged_lineage"] = "XBB.1.5*"


df.to_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full4.csv",index = None)

In [165]:
df = pd.read_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full4.csv")

for i in df.index:
    lin = df.loc[i,"lineage"] 
    full_lin = df.loc[i,"lineage_full_name"] 
    if "B.1.1.529.2.75" in full_lin:
        # df.loc[i,"lineage_full_name"] = lin
        df.loc[i,"merged_lineage"] = "BA.2.75*"
    elif "XBB.1.22.1" in full_lin:
        # df.loc[i,"lineage_full_name"] = "B.1.1.529.2.75.3.4.1.1."+lin.split("CH.")[1]
        df.loc[i,"merged_lineage"] = "XBB.1.22.1*"
    elif "B.1.1.529.5.2.1.7.14" in full_lin:
        df.loc[i,"merged_lineage"] = "BF.7.14*"
    elif "B.1.1.529.5.2.48" in full_lin:
        df.loc[i,"merged_lineage"] = "BA.5.2.48*"

df.to_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full5.csv",index = None)

In [167]:
df = pd.read_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full5.csv")

for i in df.index:
    lin = df.loc[i,"lineage"] 
    full_lin = df.loc[i,"lineage_full_name"] 
    if "B.1.1.529.5.2.1.7.14" not in full_lin and "B.1.1.529.5.2.48" not in full_lin and "B.1.1.529.5.2" in full_lin:
        df.loc[i,"merged_lineage"] = "Others"

df.to_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full6.csv",index = None)

In [168]:
df = pd.read_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full6.csv")

df["merged_lineage"].fillna("Others", inplace=True)

df.to_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/month_lineage_num_full7.csv",index = None)

In [170]:
merged_lineage_lin = {}
for i in df.index:
    rep_lin = df.loc[i,"merged_lineage"]
    sub_lin = df.loc[i,"lineage"]
    if rep_lin not in merged_lineage_lin:
        merged_lineage_lin[rep_lin] =[sub_lin]
    else:
        merged_lineage_lin[rep_lin].append(sub_lin)

In [177]:
df_tables3 = pd.DataFrame(columns=["Representative_ancestors","Representative_lineages"])
df_tables3.Representative_ancestors = merged_lineage_lin.keys()
df_tables3.Representative_lineages = merged_lineage_lin.values()
df_tables3.to_csv("/home/soniali/Desktop/02_china_recom_renew/0_raw_data/representative_lineages.csv",index = None)

In [ ]:
# 手动删除representative_lineages.csv文件“Representative_lineages”列的[]、"，调整格式